In [ ]:
from constants import FLUXES_FOLDER, SLM_FOLDER
from data_utils import load_numpy_data, flatten_data, shuffle_arrays
from plot_utils import plot_map
from modeling_utils import create_linear_architecture_for_amplitude_reconstruction, \
                           compile_linear_model_for_amplitude_reconstruction, \
                           train_linear_model_for_amplitude_reconstruction

In [ ]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"
phases_filename = "complexsine_pupphase.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"
phases_path = f"{SLM_FOLDER}/{phases_filename}"

